In [ ]:
import numpy as np
import pandas as pd
import os
import keras
from keras.utils.all_utils import Sequence

from collections import Counter, defaultdict
from pathlib import Path
from PIL import Image

import seaborn as sns
import matplotlib.pyplot as plt
import math

import warnings
warnings.filterwarnings("ignore")

print(os.listdir("../input"))

### Загрузка данных

In [ ]:
print(os.listdir("../input/severstal-steel-defect-detection"))

In [ ]:
# Два датасета. Sample_submission.csv & train.csv
df_dirname = "../input/severstal-steel-defect-detection"

train_df = pd.read_csv( os.path.join(df_dirname, "train.csv") )
sample_df = pd.read_csv( os.path.join(df_dirname, "sample_submission.csv") )

Все изображения из папок имеют одинаковый размер 1600x256

### Знакомство с данными

In [ ]:
train_df.head()

In [ ]:
sample_df.head()

In [ ]:
print(train_df["ImageId"].duplicated().any())
print(sample_df["ImageId"].duplicated().any())

В ImageId у train выборки есть повторяющиеся значения

Это означает, что у train выборки есть экземпляров с несколькими видами дефектов

In [ ]:
print(train_df["ClassId"].isnull().any())
print(sample_df["ClassId"].isnull().any())

print(train_df["EncodedPixels"].isnull().any())
print(sample_df["EncodedPixels"].isnull().any())

print(train_df["ImageId"].isnull().any())
print(sample_df["ImageId"].isnull().any())

Пропущенных значений видимо нет

Добавим те изображения в датасет, у которых нет дефектов, но которые есть в папке train_images

In [ ]:
for img_filename in os.listdir("../input/severstal-steel-defect-detection/train_images"):
    if img_filename not in list(train_df.ImageId):
        train_df = train_df.append({'ImageId': img_filename}, ignore_index=True)
train_df = train_df.sort_values("ImageId")

In [ ]:
values_counter = list(Counter(train_df.loc[train_df.ClassId.isnull() == False, "ImageId"]).values())
# output = [1, 1, 1, 1, 2, ...]
values_counter = Counter(values_counter)

values_counter[0] = len(train_df.loc[train_df.ClassId.isnull(), "ImageId"])
values_counter= dict(sorted(values_counter.items(), key=lambda i: i[0]))  
# output = {0: 5902, 1: 6239, 2: 425, 3: 2}

fig, ax = plt.subplots()
sns.barplot(x=list(values_counter.keys()), y=list(values_counter.values()), ax=ax)
ax.set_title("Кол-во дефектов в изображениях")
ax.set_xlabel("Defects number")

values_counter

In [ ]:
values_counter = Counter(train_df.loc[train_df.ClassId.isnull() == False, "ClassId"])
# output = Counter({1: 897, 3: 5150, 4: 801, 2: 247})
fig, ax = plt.subplots()
sns.barplot(x=list(values_counter.keys()), y=list(values_counter.values()), ax=ax)
ax.set_title("Кол-во изображений в каждом классе")
ax.set_xlabel("ClassId")

values_counter

### Подготовка данных

Переформатируем датасет

In [ ]:
train2 = pd.DataFrame(columns=['ImageId', 'e1', 'e2', 'e3', 'e4'])  
for row in range(len(train_df)):
    if train_df.iloc[row, 0] not in list(train2.ImageId):
        train2 = train2.append({'ImageId' : train_df.iloc[row, 0]}, ignore_index=True)
    if np.isnan(train_df.iloc[row, 1]) == False:
        train2.loc[train2['ImageId'] == train_df.iloc[row, 0], f'e{int(train_df.iloc[row, 1])}'] = train_df.iloc[row, 2]
train2.reset_index(inplace=True,drop=True)
train2.fillna('',inplace=True); 
train2['count'] = np.sum(train2.iloc[:,1:]!='',axis=1).values
train2.head()

e1 - EncodedPixels первого дефекта, e2, e3 и e4 соотвественно. Count - общее кол-во дефектов в экземпляре

#### DataGenerator

In [ ]:
class DataGenerator(Sequence):
    def __init__(self, df, batch_size = 16, subset="train", shuffle=False, 
                 preprocess=None, info={}):
        super().__init__()
        self.df = df
        self.shuffle = shuffle
        self.subset = subset
        self.batch_size = batch_size
        self.preprocess = preprocess
        self.info = info
        
        if self.subset == "train":
            self.data_path = path + 'train_images/'
        elif self.subset == "test":
            self.data_path = path + 'test_images/'
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.df) / self.batch_size))
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    
    def __getitem__(self, index): 
        X = np.empty((self.batch_size,128,800,3),dtype=np.float32)
        y = np.empty((self.batch_size,128,800,4),dtype=np.int8)
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        for i,f in enumerate(self.df['ImageId'].iloc[indexes]):
            self.info[index*self.batch_size+i]=f
            X[i,] = Image.open(str(self.data_path + f)).resize((800,128))
            if self.subset == 'train': 
                for j in range(4):
                    y[i,:,:,j] = rle2maskResize(self.df['e'+str(j+1)].iloc[indexes[i]])
        if self.preprocess!=None:
            X = self.preprocess(X)
        if self.subset == 'train':
            return X, y
        else: 
            return X

In [ ]:
# https://www.kaggle.com/titericz/building-and-visualizing-masks
def rle2maskResize(rle):
    # CONVERT RLE TO MASK 
    if (pd.isnull(rle))|(rle==''): 
        return np.zeros((128,800) ,dtype=np.uint8)
    
    height= 256
    width = 1600
    mask= np.zeros( width*height ,dtype=np.uint8)

    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]-1
    lengths = array[1::2]    
    for index, start in enumerate(starts):
        mask[int(start):int(start+lengths[index])] = 1
    
    return mask.reshape( (height,width), order='F' )[::2,::2]

def mask2contour(mask, width=3):
    # CONVERT MASK TO ITS CONTOUR
    w = mask.shape[1]
    h = mask.shape[0]
    mask2 = np.concatenate([mask[:,width:],np.zeros((h,width))],axis=1)
    mask2 = np.logical_xor(mask,mask2)
    mask3 = np.concatenate([mask[width:,:],np.zeros((width,w))],axis=0)
    mask3 = np.logical_xor(mask,mask3)
    return np.logical_or(mask2,mask3)

def mask2pad(mask, pad=2):
    # ENLARGE MASK TO INCLUDE MORE SPACE AROUND DEFECT
    w = mask.shape[1]
    h = mask.shape[0]
    
    # MASK UP
    for k in range(1,pad,2):
        temp = np.concatenate([mask[k:,:],np.zeros((k,w))],axis=0)
        mask = np.logical_or(mask,temp)
    # MASK DOWN
    for k in range(1,pad,2):
        temp = np.concatenate([np.zeros((k,w)),mask[:-k,:]],axis=0)
        mask = np.logical_or(mask,temp)
    # MASK LEFT
    for k in range(1,pad,2):
        temp = np.concatenate([mask[:,k:],np.zeros((h,k))],axis=1)
        mask = np.logical_or(mask,temp)
    # MASK RIGHT
    for k in range(1,pad,2):
        temp = np.concatenate([np.zeros((h,k)),mask[:,:-k]],axis=1)
        mask = np.logical_or(mask,temp)
    
    return mask 

### Визуализация данных с дефектами

In [ ]:
# DEFECTIVE IMAGE SAMPLES
path = "../input/severstal-steel-defect-detection/"

filenames = {}
defects = list(train2[train2['e1']!=''].sample(3).index)
defects += list(train2[train2['e2']!=''].sample(3).index)
defects += list(train2[train2['e3']!=''].sample(7).index)
defects += list(train2[train2['e4']!=''].sample(3).index)
print(defects)

# DATA GENERATOR
train_batches = DataGenerator(train2[train2.index.isin(defects)],shuffle=True,info=filenames)
print('Images and masks from our Data Generator')
print('KEY: yellow=defect1, green=defect2, blue=defect3, magenta=defect4')

# DISPLAY IMAGES WITH DEFECTS
for i,batch in enumerate(train_batches):
    plt.figure(figsize=(14,50))
    for k in range(16):
        plt.subplot(16,1,k+1)
        img = batch[0][k,]
        img = Image.fromarray(img.astype('uint8'))
        img = np.array(img)
        extra = '  has defect'
        for j in range(4):
            msk = batch[1][k,:,:,j]
            msk = mask2pad(msk,pad=3)
            msk = mask2contour(msk,width=2)
            if np.sum(msk)!=0: extra += ' '+str(j+1)
            if j==0: # yellow
                img[msk==1,0] = 235 
                img[msk==1,1] = 235
            elif j==1: img[msk==1,1] = 210 # green
            elif j==2: img[msk==1,2] = 255 # blue
            elif j==3: # magenta
                img[msk==1,0] = 255
                img[msk==1,2] = 255
        plt.title(filenames[16*i+k]+extra)
        plt.axis('off') 
        plt.imshow(img)
    plt.subplots_adjust(wspace=0.05)
    plt.show()

In [ ]:
! pip install segmentation-models

In [ ]:
import segmentation_models as sm
sm.set_framework('tf.keras')
sm.framework()

In [ ]:
from keras import backend as K
# https://www.kaggle.com/xhlulu/severstal-simple-keras-u-net-boilerplate

# COMPETITION METRIC
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [ ]:
# LOAD UNET WITH PRETRAINING FROM IMAGENET
preprocess = sm.get_preprocessing('resnet34') # for resnet, img = (img-110.0)/1.0
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=3)
model = sm.Unet('resnet34', input_shape=(128, 800, 3), classes=4, activation='sigmoid')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef])

# TRAIN AND VALIDATE MODEL
idx = int(0.8*len(train2)); print()
train_batches = DataGenerator(train2.iloc[:idx],shuffle=True,preprocess=preprocess)
valid_batches = DataGenerator(train2.iloc[idx:],preprocess=preprocess)
history = model.fit(train_batches, validation_data = valid_batches, epochs = 30, verbose=1, callbacks=[callback])

In [ ]:
 # PLOT TRAINING
plt.figure(figsize=(15,5))
plt.plot(range(history.epoch[-1]+1),history.history['val_dice_coef'],label='val_dice_coef')
plt.plot(range(history.epoch[-1]+1),history.history['dice_coef'],label='trn_dice_coef')
plt.title('Training Accuracy'); plt.xlabel('Epoch'); plt.ylabel('Dice_coef');plt.legend(); 
plt.show()

In [ ]:
model.save('UNET.h5')

# LOAD MODEL
from keras.models import load_model
model = load_model('UNET.h5',custom_objects={'dice_coef':dice_coef})

In [ ]:
def mask2RLE(binary_mask):
    pix_min=250
    rle = {'counts': [], 'size': list(binary_mask.shape)}
    counts = rle.get('counts')
    
    if np.sum(binary_mask,axis=(0,1)) < pix_min:
        counts.append(409600)
        return counts
    
    last_elem = 0
    running_length = 0

    for i, elem in enumerate(binary_mask.ravel(order='F')):
        if elem == last_elem:
            pass
        else:
            counts.append(running_length)
            running_length = 0
            last_elem = elem
        running_length += 1

    counts.append(running_length)

    return counts

In [ ]:
# PREDICT
import numpy.ma as ma

THRESHOLD=0.5
predict = pd.DataFrame(columns=['ImageId', 'EncodedPixels', 'ClassId'])
        

# PREDICT 1 BATCH TEST DATASET
test = pd.read_csv(path + 'sample_submission.csv')

for c in range(math.ceil(len(test)/256)):
    try:
        test_batches = DataGenerator(test[c*256: (c+1)*256], subset='test', batch_size=256, preprocess=preprocess)
        test_preds = model.predict_generator(test_batches,steps=1,verbose=1)
    except ValueError: # in case if dataset_size is less than batch_size 
        test_batches = DataGenerator(test[c*256:], subset='test', batch_size=len(test[c*256:]), preprocess=preprocess)
        test_preds = model.predict_generator(test_batches,steps=1,verbose=1)

    for i in range(len(test_preds)):
        test_preds2 = test_preds[i].copy()
        test_preds2[test_preds2>=THRESHOLD]=1
        test_preds2[test_preds2<THRESHOLD]=0
        for j in range(4):
            currentRLE = mask2RLE(ma.resize( test_preds2[:,:,j], ( 256, 1600)))
            if len(currentRLE) > 1:
                predict = predict.append({'ImageId'       : test.loc[c*256+i, 'ImageId'],
                                          'EncodedPixels' : ' '.join(map(str, currentRLE)),
                                          'ClassId'       : j},
                                         ignore_index=True)
predict.head()

In [ ]:
predict['ClassId'] = predict['ClassId'] + 1

In [ ]:
predict.to_csv('submission.csv', index=False)